In [31]:
import pandapower as pp
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torch
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler
import pickle

In [16]:
# Create a simple 2-bus example

class SimpleTwoBus:

    def __init__(self, P, Q, G, B, V_init, theta_init):
        '''This class creates a simple 2-bus network.'''
        self.P = P
        self.Q = Q
        self.G = G
        self.B = B
        self.V_init = V_init
        self.theta_init = theta_init
        self.net = pp.create_empty_network()
        self.create_two_bus_grid()



    def create_two_bus_grid(self):
        # Create an empty network
        # net = pp.create_empty_network()
    
        # Create two buses with initialized voltage and angle
        bus1 = pp.create_bus(self.net, vn_kv=20.0, name="Bus 1")
        bus2 = pp.create_bus(self.net, vn_kv=0.4, name="Bus 2")
    
        # Initialize voltage and angle for buses
        self.net.bus.loc[bus1, 'vm_pu'] = self.V_init[0]
        self.net.bus.loc[bus1, 'va_degree'] = self.theta_init[0]
        self.net.bus.loc[bus2, 'vm_pu'] = self.V_init[1]
        self.net.bus.loc[bus2, 'va_degree'] = self.theta_init[1]
    
        # Create a transformer between the two buses
        pp.create_transformer(self.net, bus1, bus2, std_type="0.25 MVA 20/0.4 kV")
    
        # Create a load at bus 2 with specified P and Q
        pp.create_load(self.net, bus2, p_mw=self.P, q_mvar=self.Q, name="Load")
    
        # Create an external grid connection at bus 1 with specified G and B
        pp.create_ext_grid(self.net, bus1, vm_pu=1.02, name="Grid Connection", s_sc_max_mva=self.G, rx_max=self.B)


In [18]:
# Define a neural network model
class PowerFlowNN(nn.Module):
    def __init__(self):
        super(PowerFlowNN, self).__init__()
        # self.fc1 = nn.Linear(8, 10)  # Adjust input size to match the concatenated input size
        # self.fc2 = nn.Linear(10, 10)
        # self.fc3 = nn.Linear(10, 8)  # Output size should match the number of buses * 2 (voltage magnitudes and angles)

        self.fc1 = nn.Linear(2, 1000)  # Adjust input size to match the concatenated input size
        self.fc2 = nn.Linear(1000, 1000)
        self.fc3 = nn.Linear(1000, 14)  # Output size should match the number of buses

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [28]:
# class that generates and saves a dataset using runpp newton-raphson power flow

class PowerFlowDataset(Dataset):
    def __init__(self, base_network, num_samples=1000, max_iteration=50, tolerance_mva=1e-8):
        """
        Initialize the dataset with a base network and number of samples.
       
        Parameters:
        base_network (pandapowerNet): The base pandapower network.
        num_samples (int): Number of samples to generate.
        """
        self.base_net = base_network.deepcopy()  # Ensure base network is not modified
        self.num_samples = num_samples
        self.samples = []
        self.scaler_input = StandardScaler()
        self.scaler_output = StandardScaler()
        self.max_iteration = max_iteration
        self.tolerance_mva = tolerance_mva
 
        self.generate_samples()
        self.normalize_samples()
 
    def generate_samples(self):
        """
        Generate samples by first running normal power flow and then perturbing it to create ill-conditioning.
        """
        # Run a normal power flow first
        net = self.base_net.deepcopy()
        try:
            pp.runpp(net, max_iteration=100)  # Solve with standard conditions
            print("Base case solved successfully.")
        except pp.powerflow.LoadflowNotConverged:
            print("Base case did not converge. Check the network setup.")
            return
       
        # Extract the normal solution
        v_nominal = net.res_bus.vm_pu.values  # Nominal voltage magnitudes
        theta_nominal = net.res_bus.va_degree.values  # Nominal voltage angles
       
        for _ in range(self.num_samples):
            net_ill = self.base_net.deepcopy()  # Keep the network unchanged
 
            # --- Create an ill-conditioned case ---
            v_ill = v_nominal + np.random.uniform(-0.15, 0.15, len(v_nominal))  # Small perturbation
            theta_ill = theta_nominal + np.random.uniform(-30, 30, len(theta_nominal))  # Large phase shift
            # p_ill = net_ill.res_bus.p_mw.values + np.random.uniform(-200, 200, len(v_nominal))  # Large power mismatch
 
            try:
                # Re-run power flow with ill-conditioned initialization
                pp.runpp(net_ill,
                         init="auto",
                         init_vm_pu=v_ill,
                         init_va_degree=theta_ill,
                         max_iteration=self.max_iteration,
                         tolerance_mva=self.tolerance_mva)
               
                iterations = net_ill._ppc["iterations"]
                print(f"Sample {_}: Converged in {iterations} iterations")
 
                # Extract ill-conditioned solution
                Ybus = net_ill._ppc["internal"]["Ybus"].toarray()
                S = net_ill._ppc["internal"]["Sbus"]
                it = net._ppc["iterations"]
                et = net._ppc["et"]
                V_mag = net_ill.res_bus.vm_pu.values
                V_ang = net_ill.res_bus.va_degree.values
               
                # self.samples.append({
                #     "input": np.concatenate([Ybus.real.flatten(), Ybus.imag.flatten(), S.real, S.imag]),
                #     "output": np.concatenate([V_mag, V_ang])
                # })
 
                self.samples.append({
                                "input": np.concatenate([Ybus.real.flatten(),
                                                        Ybus.imag.flatten(),
                                                        S.real,
                                                        S.imag]),
                                "output": np.concatenate([V_mag, V_ang]),
                                "iteration": it,
                                "residual error": et,
                            })

            except pp.powerflow.LoadflowNotConverged:
                print(f"Sample {_}: Ill-conditioned case did not converge!")
        
        with open( "data.pkl", "wb") as f:
            pickle.dump(self.samples, f)

        
 
    def normalize_samples(self):
        """
        Normalize the input features and target values.
        """
        if not self.samples:
            raise ValueError("No valid samples generated!")
 
        inputs = np.array([sample["input"] for sample in self.samples])
        outputs = np.array([sample["output"] for sample in self.samples])
 
        self.scaler_input.fit(inputs)
        self.scaler_output.fit(outputs)
 
        for sample in self.samples:
            sample["input"] = self.scaler_input.transform([sample["input"]])[0]
            sample["output"] = self.scaler_output.transform([sample["output"]])[0]
 
    def __len__(self):
        return len(self.samples)
 
    def __getitem__(self, idx):
        sample = self.samples[idx]
        return {
            'input': torch.FloatTensor(sample['input']),
            'output': torch.FloatTensor(sample['output'])
        }
 

# generate the data

In [29]:
# Generate dataset based on given initial values

P = 0.1  # Active power in MW
Q = 0.05  # Reactive power in MVar
G = 100  # Short-circuit power in MVA
B = 0.1  # Short-circuit impedance
V_init = [1.02, 1.0]  # Initial voltages in pu
theta_init = [0, 0]  # Initial angles in degrees

# create network object
Net = SimpleTwoBus(P,Q,G,B,V_init,theta_init)
net = Net.net

# generate data
PF_data = PowerFlowDataset(net, num_samples=10, max_iteration=50, tolerance_mva=1e-8)

Base case solved successfully.
Sample 0: Converged in 4 iterations
Sample 1: Converged in 4 iterations
Sample 2: Converged in 4 iterations
Sample 3: Converged in 4 iterations
Sample 4: Converged in 4 iterations
Sample 5: Converged in 4 iterations
Sample 6: Converged in 5 iterations
Sample 7: Converged in 4 iterations
Sample 8: Converged in 4 iterations
Sample 9: Converged in 4 iterations


below this was all prior work

In [26]:
# Function to perform power flow calculation using Newton-Raphson method
# def power_flow_calculation(net):

#     pp.runpp(net, algorithm='nr', tolerance_mva=1e-8, max_iteration = 1000)
#     iterations = net._ppc["iterations"]

    
#     return np.concatenate((net.res_bus.vm_pu.values, net.res_bus.va_degree.values)), iterations

# # Generate training data
# def generate_training_data(net, num_samples=1000):
#     inputs = []
#     targets = []
#     iterations = []
#     for _ in range(num_samples):
#         # Randomly perturb the load values
#         net.load.p_mw += (np.random.rand(len(net.load)) - 0.5) * 0.1
#         net.load.q_mvar += (np.random.rand(len(net.load)) - 0.5) * 0.1
        
#         # Perform power flow calculation
#         target, k = power_flow_calculation(net)
        
#         # Store the input and target values
#         input_values = np.concatenate((net.load.p_mw.values, net.load.q_mvar.values))
#         inputs.append(input_values)
#         targets.append(target)
#         iterations.append(k)
    
#     inputs = torch.tensor(inputs, dtype=torch.float32)
#     targets = torch.tensor(targets, dtype=torch.float32)
    
#     return inputs, targets, iterations

In [22]:
def train_network_model(model, num_epochs, optimizer, criterion, targets, inputs):

    # Train the neural network model
    # num_epochs = 100
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        if (epoch+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

    print("Training completed.")

# Compare the predicted values of the neural network model with the real network
def compare_predictions_with_real_network(net, model):
    # Perform power flow calculation on the real network
    real_values = power_flow_calculation(net)
    
    # Prepare input data for the neural network model
    input_values = np.concatenate((net.load.p_mw.values, net.load.q_mvar.values))
    input_tensor = torch.tensor(input_values, dtype=torch.float32).unsqueeze(0)  # Add batch dimension
    
    # Get predictions from the neural network model
    model.eval()
    with torch.no_grad():
        predicted_values = model(input_tensor).squeeze().numpy()
    
    # Split predicted values into voltage magnitudes and angles
    predicted_vm_pu = predicted_values[:len(net.res_bus)]
    predicted_va_degree = predicted_values[len(net.res_bus):]

    difference_va_degree = predicted_va_degree - real_values[7:]
    difference_vm_pu = predicted_vm_pu - real_values[0:7]
    
    # Print predicted voltage angles for comparison
    print("Predicted Voltage Angles: ", predicted_va_degree)

    print("Reference voltage angles:", real_values[7:])

    print("Predicted Voltage Magnitudes: ", predicted_vm_pu)

    print("Reference voltage magnitudes: ", real_values[0:7])

    print("Diffefrences:")

    print(f"{difference_va_degree=}")
    print(f"{difference_vm_pu=}")

    


# run experiments

In [23]:
# Create a simple power network using pandapower
# net = pp.networks.example_simple()
# Example usage
P = 0.1  # Active power in MW
Q = 0.05  # Reactive power in MVar
G = 100  # Short-circuit power in MVA
B = 0.1  # Short-circuit impedance
V_init = [1.02, 1.0]  # Initial voltages in pu
theta_init = [0, 0]  # Initial angles in degrees
Net = SimpleTwoBus(P,Q,G,B,V_init,theta_init)
net = Net.net

# Initialize the neural network model
model = PowerFlowNN()

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Generate training data
inputs, targets, iterations = generate_training_data(net, num_samples = 100)
# Print shapes of inputs and targets to verify
print("Inputs shape: ", inputs.shape)
print("Targets shape: ", targets.shape)
print(f"{iterations=}")





Inputs shape:  torch.Size([100, 2])
Targets shape:  torch.Size([100, 4])
iterations=[3, 3, 3, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 3, 2, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4]


In [24]:
num_epochs = 100

train_network_model(model, num_epochs, optimizer, criterion, targets, inputs)

/Users/lindsayspoor/Library/Mobile Documents/com~apple~CloudDocs/Documents/PhD/Projects/ictwi_alliander/.venv/lib/python3.9/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([100, 4])) that is different to the input size (torch.Size([100, 14])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (14) must match the size of tensor b (4) at non-singleton dimension 1

In [21]:

# Compare predictions with real network values
compare_predictions_with_real_network(net, model)

Predicted Voltage Angles:  [ 47.25534   47.39404   47.354042 -93.285995 -93.3258   -93.27554
 -93.15478 ]
Reference voltage angles: [ 50.          50.0346054   50.0346054  -98.110326   -98.110326
 -98.04583138 -97.94416185]
Predicted Voltage Magnitudes:  [0.961529   0.95510197 0.9513805  0.9575139  0.9274514  0.9759572
 0.95710087]
Reference voltage magnitudes:  [1.02       1.02084103 1.02084103 1.02451121 1.02451121 1.03
 1.02332043]
Diffefrences:
difference_va_degree=array([-2.74465942, -2.64056624, -2.68056334,  4.82433051,  4.78452796,
        4.77028816,  4.78938432])
difference_vm_pu=array([-0.05847098, -0.06573906, -0.06946054, -0.06699728, -0.09705984,
       -0.05404279, -0.06621956])


# testing stuff

In [ ]:
pp.runpp(net, algorithm='nr')

# Prepare input
Ybus = net._ppc["internal"]["Ybus"].toarray()
S = net._ppc["internal"]["Sbus"]
input_tensor = torch.FloatTensor(np.concatenate([
    Ybus.real.flatten(), 
    Ybus.imag.flatten(),
    S.real, 
    S.imag
]))

V_mag_ref = net.res_bus.vm_pu.values 
V_ang_ref = net.res_bus.va_degree.values

print("Reference voltage magnitudes:", V_mag_ref)
print("Reference voltage angles:", V_ang_ref)

amir's stuff

In [ ]:
import pandapower as pp
import numpy as np
import torch
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler
 
class PowerFlowDataset(Dataset):
    def __init__(self, base_network, num_samples=1000):
        """
        Initialize the dataset with a base network and number of samples.
       
        Parameters:
        base_network (pandapowerNet): The base pandapower network.
        num_samples (int): Number of samples to generate.
        """
        self.base_net = base_network.deepcopy()  # Ensure base network is not modified
        self.num_samples = num_samples
        self.samples = []
        self.scaler_input = StandardScaler()
        self.scaler_output = StandardScaler()
 
        self.generate_samples()
        self.normalize_samples()
 
    def generate_samples(self):
        """
        Generate samples by first running normal power flow and then perturbing it to create ill-conditioning.
        """
        # Run a normal power flow first
        net = self.base_net.deepcopy()
        try:
            pp.runpp(net, max_iteration=100)  # Solve with standard conditions
            print("Base case solved successfully.")
        except pp.powerflow.LoadflowNotConverged:
            print("Base case did not converge. Check the network setup.")
            return
       
        # Extract the normal solution
        v_nominal = net.res_bus.vm_pu.values  # Nominal voltage magnitudes
        theta_nominal = net.res_bus.va_degree.values  # Nominal voltage angles
       
        for _ in range(self.num_samples):
            net_ill = self.base_net.deepcopy()  # Keep the network unchanged
 
            # --- Create an ill-conditioned case ---
            v_ill = v_nominal + np.random.uniform(-0.15, 0.15, len(v_nominal))  # Small perturbation
            theta_ill = theta_nominal + np.random.uniform(-30, 30, len(theta_nominal))  # Large phase shift
            # p_ill = net_ill.res_bus.p_mw.values + np.random.uniform(-200, 200, len(v_nominal))  # Large power mismatch
 
            try:
                # Re-run power flow with ill-conditioned initialization
                pp.runpp(net_ill,
                         init="auto",
                         init_vm_pu=v_ill,
                         init_va_degree=theta_ill,
                         max_iteration=50)
               
                iterations = net_ill._ppc["iterations"]
                print(f"Sample {_}: Converged in {iterations} iterations")
 
                # Extract ill-conditioned solution
                Ybus = net_ill._ppc["internal"]["Ybus"].toarray()
                S = net_ill._ppc["internal"]["Sbus"]
                V_mag = net_ill.res_bus.vm_pu.values
                V_ang = net_ill.res_bus.va_degree.values
               
                self.samples.append({
                    "input": np.concatenate([Ybus.real.flatten(), Ybus.imag.flatten(), S.real, S.imag]),
                    "output": np.concatenate([V_mag, V_ang])
                })
 
            except pp.powerflow.LoadflowNotConverged:
                print(f"Sample {_}: Ill-conditioned case did not converge!")
 
    def normalize_samples(self):
        """
        Normalize the input features and target values.
        """
        if not self.samples:
            raise ValueError("No valid samples generated!")
 
        inputs = np.array([sample["input"] for sample in self.samples])
        outputs = np.array([sample["output"] for sample in self.samples])
 
        self.scaler_input.fit(inputs)
        self.scaler_output.fit(outputs)
 
        for sample in self.samples:
            sample["input"] = self.scaler_input.transform([sample["input"]])[0]
            sample["output"] = self.scaler_output.transform([sample["output"]])[0]
 
    def __len__(self):
        return len(self.samples)
 
    def __getitem__(self, idx):
        sample = self.samples[idx]
        return {
            'input': torch.FloatTensor(sample['input']),
            'output': torch.FloatTensor(sample['output'])
        }
 
 
 
net = pp.networks.example_simple()
PF_data = PowerFlowDataset(net, num_samples=10